# Study of Lending Club loans

## Introduction

Lending Club is an online credit marketplace, facilitating personal loans, business loans, and financing for elective medical procedures. Lending Club has provided datasets that contain complete loan data for all loans issued through the specific time period stated, including the current loan status (Current, Late, Fully Paid, etc.) , latest payment information and the financial and credit rating information about the loan applicant collected by Lending Club.

## Objective

 The Capstone project aims to learn the payment patterns of applicants and build a predictive model that will predict the repayment class of the loan. For this study, the loan repayment type has been classified into three as follows
 
  - **Good loans** - The installments will be always on time and the loan will be fully paid on or before the end of loan term.
  - **Bad loans** - The loan will end up in Charge Off and LendingClub will end up in spending for collection and recoveries process.
  - **Delinquent loans** - The loan installments will be delayed one or more times but not likely to end up in charge off.
  
## Data

 We will be considering the LendingClub dataset for the year 2014. At a high level, the dataset contains the below attribute groups. 
 
  - *Applicant Identity* - Basic Identity information about the applicant like home state, employment length, home-ownership type etc
  - *Applicant Income* - The reported annual income as provided by the applicant and the verification status
  - *Loan* - Attributes directly related to the loan like the loan purpose, amount, interest rate, status, payment of various fees etc
  - *Credit History* - Attributes collected about the applicant's credit history like no. of bankcard accounts, mortgage accounts, credit balances, debt to income ratio etc
  - *Delinquency History* - Attributes collected about the applicant's previous delinquent payments like no. of chargeoffs / collections, no. of accounts past due payment etc.
  
  
  - The *loan_status* attribute can be one among the following.
      - 'Fully Paid'
      - 'Current'
      - 'In Grace Period'
      - 'Late (16-30 days)'
      - 'Late (31-120 days)'
      - 'Default'
      - 'Charged Off'
  
    The detailed list of attributes can be viewed [ here ] ('https://github.com/anurekhat/Capstone/blob/master/LendingClub/LCDataDictionary2014.xlsx '). 
  

###  Data Clean ups, filtering and transformations
 
 
  - The below attributes have been dropped for further analysis as they are manual text entries that serves more as just informational. 
      - *emp_title*
      - *desc*
      - *title*
      
      
  - There are two available terms for loans - 36 months and 60 months. For this study, we consider only the 36 months term loans as the 60 months term loans initiated in 2014 will be mostly still running as on 2017 and including them may skew the features of 'Charged Off' loans. 
  
  
  - The loan records with *total_rec_late_fee* as a non-zero value have been marked as delinquent. i.e If the applicant has ever paid a late fee, that loan is marked as delinquent. Please see the explanation on **Classification** below
  
  - The below date attributes have been converted to the appropriate duration attribute, considering Apr 2017 as the reference date
      - *issue_d* converted to *mo_sin_loan_funded* (months since loan was funded)
      - *earliest_cr_line* converted to *mo_sin_earliest_cr_line* (months since the earliest credit line was opened)
      - *last_pymnt_d* converted to *mo_sin_last_pymnt* (months since the last payment)
      - *last_credit_pull_d* converted to *mo_sin_last_credit_pull* (months since the last credit pull by LC)
      
  - **Imputing Missing Values**
      - *last_pymnt_d* (*mo_sin_last_pymnt*) : This attribute is expected to be null when there has not been any installments that were paid. The date is set to May 2017 so that the duration attribute holds the value -1 and these records are considered separate by the model estimator
      - *last_credit_pull_d* (*mo_sin_last_credit_pull*) : This attribute is expected to be null when LC has not pulled the credit info of the applicant. The date is set to May 2017 so that the duration attribute holds the value -1 and these records are considered separate by the model estimator
      - *mths_since_last_delinq* : This attribute is expected to be null when the applicant has no prior delinquencies. Imputing these to -1 to set these apart
      - *mths_since_last_record* : This attribute is expected to be null when the applicant has no public records. Imputing these to -1 to set these apart
      - *mths_since_last_major_derog* : This attribute is expected to be null when the applicant has no previous derogatory records. Imputing these to -1 to set these apart
      - *mo_sin_old_il_acct* : This attribute is expected to be null when the applicant has no bank installment accounts. Imputing these to -1 to set these apart
      - *mths_since_recent_bc_dlq* : This attribute is expected to be null when there's no previous bankcard delinquency. Imputing these to -1 to set these apart
      - *mths_since_recent_inq* : This attribute is expected to be null when there's no previous inquiries. Imputing these to -1 to set these apart
      - *mths_since_recent_revol_delinq* : This attribute is expected to be null when there's no previous revolving account delinquencies. Imputing these to -1 to set these apart
      - *num_tl_120dpd_2m* : This attribute is expected to be null when there's no accounts currently 120 days past due. Imputing these to 0
      - *mths_since_recent_bc* : This attribute is expected to be null when info not available. Imputing these to the median
      - *bc_open_to_buy* : This attribute is expected to be null when info is not available. Imputing these to the median
      - *bc_util* : This attribute is expected to be null when info is not available. Imputing these to the median
      - *avg_cur_bal* : This attribute is expected to be null when info is not available. Imputing these to the median
      - *percent_bc_gt_75* : This attribute is expected to be null when info is not available. Imputing these to the median
      - *revol_util* : This attribute is expected to be null when info is not available. Imputing these to the median
      
  - **Categorical features** 
      Categorical features have been converted to the 'One Hot Encoding' format using pandas `get_Dummies()` method
      
  - The following attributes are not being considered as the 'X' independent variables in the model as these are not expected to impact the outcome variable of loan_class
      - *term* : As it will be always 36 months
      - *zip_code* : The 3 digit coded value is not expected to impact outcome
      - *loan_status* : This is more part of the outcome than an input variable
      - *application_type* : As it will be always 'INDIVIDUAL'
      - *id* : Identifier 
      - *next_pymnt_d* : Not expected to impact outcome

      
      
#### Classification
   1. All loan records where *loan_status* is 'Fully Paid' or 'Current' and *total_rec_late_fee* is zero are classified as **Good**
      
      
   2. All loan records with *loan_status* 'Charged Off' are classified as **Bad**
      
      
   3. All loan records that have a non-zero value for *total_rec_late_fee* or loan_status is one among { 'In Grace Period', 'Late (16-30 days)', 'Late (31-120 days)', 'Default'} is classified as **Delinquent**. Note that there could be some >Charged Off loans that could have been also delinquent. The Charged off status will take priority in such cases and those loans will be classified as **Bad** 
      
  
## Methodology

The cleaned up dataset has been divided into training (70%) and test (30%) datasets. As a first step, a simple logistic regression model has been built using the training dataset. 
 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as st
import statsmodels.stats.weightstats as wst
%matplotlib inline
pd.options.display.max_columns = None
pd.options.display.max_rows = None
plt.rcParams['figure.figsize'] = (10.0, 8.0)

In [2]:
# Read the dataset.

# Define a function for string % convertion to a float ratio
def p2f(x):
    perc = None
    if (x != None) :
        if (type(x) is str) :
            x = x.strip('%')
        perc = float(x)/100
    return perc

loan_data = pd.read_csv('./LoanStats2014.csv', skiprows=1, skipfooter=2, parse_dates = [15, 26, 45,47, 48], 
                        infer_datetime_format = True, engine = 'python', converters = {'int_rate' : p2f, 'revol_util' : p2f})

In [3]:
#Removing the attribute columns that have all null values. Only the attributes listed in the above DataDictionary will remain.
loan_data.dropna(axis = 1, how = 'all', inplace= True)

#Since id values are not provided in the dataset, create id variable from the index.
loan_data['id'] = loan_data.index


In [4]:
loan_data['term_mnths'] = loan_data.term.str.strip('months').astype(int)

In [5]:
len(loan_data)

235629

In [6]:
#Filtering out the 36 months term loans
loan_data = loan_data[loan_data.term_mnths == 36]
len(loan_data)

162570

In [7]:
#Dropping textual columns from the dataset
loan_data.drop(['emp_title', 'desc', 'title'], axis = 1, inplace = True)

In [8]:
#Marking loans that had at least one late payment
loan_data['late_fee_rec_indicator'] = (loan_data.total_rec_late_fee > 0)


In [9]:
#loans across different loan statuses
#sns.countplot(x = 'loan_status', hue = 'late_fee_rec_indicator', data = loan_data)

In [10]:
# Classify the loans as 'Good' ,'Bad', 'Delinquent'

def classify_loan(row) :
    val = ''
    if ((row.loan_status in ['Fully Paid', 'Current']) & (row.late_fee_rec_indicator == False) ) :
        val = 'Good'
    elif row.loan_status in ['Charged Off'] :
        val = 'Bad'
    elif ( (row.loan_status not in ['Fully Paid', 'Current', 'Charged Off']) | 
         row.late_fee_rec_indicator == True) :
        val = 'Delinquent'
    return val

loan_data['loan_class'] = loan_data.apply(classify_loan, axis = 1)

In [11]:
pd.DataFrame( loan_data.groupby(['loan_class', 'loan_status','late_fee_rec_indicator']).size())

0
loan_class loan_status        late_fee_rec_indicator       
Bad        Charged Off        False                   17453
                              True                     1927
Delinquent Current            True                      956
           Default            False                       3
                              True                        2
           Fully Paid         True                     1076
           In Grace Period    False                    1015
                              True                      110
           Late (16-30 days)  False                     300
                              True                       63
           Late (31-120 days) False                    1276
                              True                      217
Good       Current            False                   48236
           Fully Paid         False                   89936

In [12]:
#Transform date fields into appropriate duration units
from datetime import datetime
loans = loan_data.copy()
ref_datetime = datetime.strptime('2017-04-02', '%Y-%m-%d')
# Issue date
loans['mo_sin_loan_funded'] = ((ref_datetime - loans['issue_d']) / np.timedelta64(1, 'M')).astype(int)

# Earliest credit line
loans['mo_sin_earliest_cr_line'] = ((ref_datetime - loans['earliest_cr_line']) / np.timedelta64(1, 'M')).astype(int)

In [13]:
#Imputing missing values and converting to duration units
#Last payment date is null when no installments have been made. Setting the nulls to May 2017, so that the calculated duration
# field will have the value -1
loans['last_pymnt_d'] = loans['last_pymnt_d'].fillna(pd.to_datetime('2017-05-02'))
loans['mo_sin_last_pymnt'] = ((ref_datetime - loans['last_pymnt_d']) / np.timedelta64(1, 'M')).astype(int)

#last_credit_pull_d
#Setting the nulls to May 2017, so that the calculated duration field will have the value -1
loans['last_credit_pull_d'] = loans['last_credit_pull_d'].fillna(pd.to_datetime('2017-05-02'))
loans['mo_sin_last_credit_pull'] = ((ref_datetime - loans['last_credit_pull_d']) / np.timedelta64(1, 'M')).astype(int)

In [15]:
# Imputing missing values

#mths_since_last_delinq
# this will be null when the applicant has no prior delinquencies. Imputing these to -1 to set these apart.
loans['mths_since_last_delinq'] = loans['mths_since_last_delinq'].fillna(-1)

#mths_since_last_record
# this will be null when the applicant has no public records. Imputing these to -1 to set these apart.
loans['mths_since_last_record'] = loans['mths_since_last_record'].fillna(-1)

#mths_since_last_major_derog
# this will be null when the applicant has no previous derogatory records. Imputing these to -1 to set these apart.
loans['mths_since_last_major_derog'] = loans['mths_since_last_major_derog'].fillna(-1)

#mo_sin_old_il_acct
# this will be null when the applicant has no bank installment accounts. Imputing these to -1 to set these apart.
loans['mo_sin_old_il_acct'] = loans['mo_sin_old_il_acct'].fillna(-1)

#mths_since_recent_bc_dlq
# this will be null when there's no previous bankcard delinquency. Imputing these to -1 to set these apart.
loans['mths_since_recent_bc_dlq'] = loans['mths_since_recent_bc_dlq'].fillna(-1)

#mths_since_recent_inq
# this will be null when there's no previous inquiries. Imputing these to -1 to set these apart.
loans['mths_since_recent_inq'] = loans['mths_since_recent_inq'].fillna(-1)

#mths_since_recent_revol_delinq
# this will be null when there's no previous revolving account delinquencies. Imputing these to -1 to set these apart.
loans['mths_since_recent_revol_delinq'] = loans['mths_since_recent_revol_delinq'].fillna(-1)

#num_tl_120dpd_2m
# this will be null when there's no accounts currently 120 days past due. Imputing these to 0.
loans['num_tl_120dpd_2m'] = loans['num_tl_120dpd_2m'].fillna(0)

#mths_since_recent_bc
# this will be null when info not available. Imputing these to the median
loans['mths_since_recent_bc'] = loans['mths_since_recent_bc'].fillna(loans.mths_since_recent_bc.median())

#bc_open_to_buy
# this will be null when info is not available. Imputing these to the median
loans['bc_open_to_buy'] = loans['bc_open_to_buy'].fillna(loans.bc_open_to_buy.median())

#bc_util
# this will be null when info is not available. Imputing these to the median
loans['bc_util'] = loans['bc_util'].fillna(loans.bc_util.median())

#avg_cur_bal
# this will be null when info is not available. Imputing these to the median
loans['avg_cur_bal'] = loans['avg_cur_bal'].fillna(loans.avg_cur_bal.median())

#percent_bc_gt_75
# this will be null when info is not available. Imputing these to the median
loans['percent_bc_gt_75'] = loans['percent_bc_gt_75'].fillna(loans.percent_bc_gt_75.median())

#revol_util
# this will be null when info is not available. Imputing these to the median
loans['revol_util'] = loans['revol_util'].fillna(loans.revol_util.median())

In [16]:
#Drop the now unused columns
loans.drop(['issue_d', 'earliest_cr_line', 'last_pymnt_d', 'last_credit_pull_d'], axis = 1, inplace = True)


In [17]:
# Convert categorical features to One Hot Encode format using Pandas getDummies
categorical_features = ['grade', 'sub_grade', 'home_ownership','emp_length', 'verification_status', 'pymnt_plan', 'purpose', 'addr_state',
                       'initial_list_status']
loans_with_dummies = pd.get_dummies(columns = categorical_features, data= loans)

In [31]:
# Implement a simple logistic regression

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

# Identifying X values - Removing all derived columns, date columns, id and constants
X = loans_with_dummies.drop(['term', 'zip_code','loan_status', 'application_type','id','term_mnths', 'late_fee_rec_indicator', 
                             'loan_class', 'next_pymnt_d' ], axis = 1)
loans_with_dummies['loan_class'] = loans_with_dummies.loan_class.astype('category')
Y = loans_with_dummies.loan_class.cat.codes


In [32]:
# Split the data into a training and test set.
Xlr, Xtestlr, ylr, ytestlr = train_test_split(X.values, Y.values, test_size = 0.30)

print(Xlr.shape)
print (ylr.shape)

print(Xtestlr.shape)
print(ytestlr.shape)

#Using the class_weight parameter as balanced so that the algo adjusts for the less frequent loan classes of bad and delinquent
clf = LogisticRegression(class_weight= 'balanced')
# Fit the model on the trainng data.
clf.fit(Xlr, ylr)


(113799, 201)
(113799,)
(48771, 201)
(48771,)


LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [34]:
# Accuracy score
from sklearn.metrics import accuracy_score

# Print the accuracy from the test data.
print('Accuracy score from learning dataset',accuracy_score(ylr, clf.predict(Xlr) ))

# Print the accuracy from the test data.
print('Accuracy score from testing dataset',accuracy_score(ytestlr, clf.predict(Xtestlr)))

Accuracy score from learning dataset 0.959920561692
Accuracy score from testing dataset 0.961021918763


In [41]:
# Log loss
from sklearn.metrics import log_loss

# Print the log loss from the test data.
print('Log loss from learning dataset',log_loss(ylr, clf.predict_proba(Xlr)))

# Print the log loss from the test data.
print('Log loss from testing dataset',log_loss(ytestlr, clf.predict_proba(Xtestlr)))


Log loss from learning dataset 0.244815742657
Log loss from testing dataset 0.243824316919


In [44]:
# ROC AUC score
from sklearn.metrics import roc_auc_score

# Print the ROC AUC score from the test data.
print('ROC AUC score from learning dataset',log_loss(ylr, clf.decision_function(Xlr)))

# Print the ROC AUC score from the test data.
print('ROC AUC score from testing dataset',log_loss(ytestlr, clf.decision_function(Xtestlr)))



ROC AUC score from learning dataset 0.878122996468
ROC AUC score from testing dataset 0.831897583324


In [47]:
# Confusion Matrix
from sklearn.metrics import confusion_matrix

print ('Learing dataset: \n')
print (confusion_matrix(ylr, clf.predict(Xlr)))

print ('Test dataset: \n')
print (confusion_matrix(ytestlr, clf.predict(Xtestlr)))

Learing dataset: 

[[13525     0    43]
 [    0   311  3238]
 [    0  1280 95402]]
Test dataset: 

[[ 5799     1    12]
 [    0   135  1334]
 [    1   553 40936]]
